In [1]:
from kafka import KafkaProducer
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd
import json

import tkinter as tk

import requests
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [2]:
#Retourne le code source de l'URL qui est passé en paramètre

def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [3]:
#Retourne un dataFrame qui contient le contenu du flux RSS passé en paramètre

def get_feed(url):   
    response = get_source(url)
    
    df = pd.DataFrame(columns = ['title', 'pubDate', 'link', 'description'])

    with response as r:
        items = r.html.find("item", first=False)

        for item in items:        

            title = item.find('title', first=True).text
            pubDate = item.find('pubDate', first=True).text
            link = item.find('guid', first=True).text
            description = item.find('description', first=True).text


            row = {'title': title, 'pubDate': pubDate, 'link': link, 'description': description}
            df = df.append(row, ignore_index=True)

    return df

In [4]:
#Permet à l'utilisateur de sélectionner les sites internet qu'il veut consulter parmi une liste prédéfinie
#Retourne une liste qui contient ces sites 
def ouvrir_fenetre():
    sites = {
        "Le Monde": "https://www.lemonde.fr/rss/une.xml",
        "OVH Cloud": "https://blog.ovhcloud.com/feed/",
        "Actu IA": "https://www.actuia.com/feed/",
        "20 Minutes": "https://www.20minutes.fr/feeds/rss-une.xml",
        "France 24":"https://www.france24.com/fr/rss"
    }

    def ajouter_site():
        selection.clear()
        for site in listebox.curselection():
            nom_site = listebox.get(site)
            selection.append(sites[nom_site])
        fenetre.quit()

    fenetre = tk.Tk()

    listebox = tk.Listbox(fenetre, selectmode=tk.MULTIPLE)
    for site in sites:
        listebox.insert(tk.END, site)

    bouton_ajouter = tk.Button(fenetre, text="Ajouter", command=ajouter_site)

    selection = []

    listebox.grid(row=0, column=0)
    bouton_ajouter.grid(row=1, column=0)

    fenetre.mainloop()

    return selection

In [6]:
#Appel de la fonction
urls=ouvrir_fenetre()
urls

['https://www.lemonde.fr/rss/une.xml', 'https://www.actuia.com/feed/']

In [7]:
# Création du dataFrame en fonction des sites sélectionnés
dfs = []
for url in urls:
    df = get_feed(url)
    dfs.append(df)

# Fusion des dataFrame en un seul
data = pd.concat(dfs, ignore_index=True)


/Users/antoinesusini/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/var/folders/kb/kq8rbcl93hq1t3bmnp4b0yj00000gn/T/ipykernel_59359/664555981.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/var/folders/kb/kq8rbcl93hq1t3bmnp4b0yj00000gn/T/ipykernel_59359/664555981.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [12]:
#Nettoyage des données 

data['title'] = data['title'].apply(lambda x: x.replace('ê', 'e'))
data['title'] = data['title'].apply(lambda x: x.replace("'", ""))
data['description'] = data['description'].apply(lambda x: x.replace("'", ""))
for i in range(len(data)):
    data['description'] = data['description'].apply(lambda x: x.replace('ê', 'e'))
    if '[' in data.loc[i,'title'] and ']' in data.loc[i,'title']:
        data.loc[i,'title'] = data.loc[i,'title'].split('<![CDATA[')[1].split(']')[0] 
    if '[' in data.loc[i,'description'] and ']' in data.loc[i,'description']:
        data.loc[i,'description'] = data.loc[i,'description'].split('<![CDATA[')[1].split(']')[0] 
    else:
        data.loc[i,'title'] = data.loc[i,'title']
        data.loc[i,'description'] = data.loc[i,'description']


In [13]:
#Classement des articles par ordre décroissant (plus récents en premier)
data['pubDate'] = pd.to_datetime(data['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')

data = data.sort_values('pubDate', ascending=False)

data = data.reset_index(drop=True)

In [14]:
data

,title,pubDate,link,description
0,Kylian Mbappé conteste publiquement une campag...,2023-04-06 17:44:54+02:00,https://www.lemonde.fr/sport/article/2023/04/0...,Après plusieurs conflits liés à l’utilisation ...
1,Mesures anti-inflation : le blocage des prix f...,2023-04-06 17:08:02+02:00,https://www.lemonde.fr/economie/article/2023/0...,Les députés ont repoussé l’ensemble d’une prop...
2,"Qu’est ce que le métabolite du chlorothalonil,...",2023-04-06 17:00:05+02:00,https://www.lemonde.fr/planete/live/2023/04/06...,"Selon un rapport de l’Anses, la concentration ..."
3,Chantier du Grand Paris Express : un cinquième...,2023-04-06 16:27:28+02:00,https://www.lemonde.fr/economie/article/2023/0...,Une cellule médico-psychologique a été mise en...
4,La vidéosurveillance « intelligente » sera fin...,2023-04-06 16:21:45+02:00,https://www.lemonde.fr/societe/article/2023/04...,"Réunis en commission mixte paritaire, députés ..."
5,Paris 2024 : des épreuves pour tester l’organi...,2023-04-06 16:20:34+02:00,https://www.lemonde.fr/sport/article/2023/04/0...,Les organisateurs des Jeux olympiques et paral...
6,Une trentaine de roquettes tirées sur Israël d...,2023-04-06 16:02:41+02:00,https://www.lemonde.fr/international/article/2...,"Selon l’agence de presse officielle libanaise,..."
7,Emmanuel Macron demande au président chinois X...,2023-04-06 15:57:42+02:00,https://www.lemonde.fr/international/article/2...,"Le président français, qui a plaidé pour « une..."
8,"Silvio Berlusconi, actuellement en soins inten...",2023-04-06 15:55:27+02:00,https://www.lemonde.fr/international/article/2...,"Selon l’agence de presse Ansa, qui cite son en..."
9,Tensions à « Paris Match » : une membre de la ...,2023-04-06 15:53:55+02:00,https://www.lemonde.fr/economie/article/2023/0...,"Selon les syndicats, la journaliste visée a re..."


In [10]:
#Connexion à Kafka et Cassandra

# Configure Kafka
bootstrap_servers = ['localhost:9092']
topic = 'your_kafka_topic'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

# Configure Cassandra
contact_points = ['localhost']
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(contact_points=contact_points, auth_provider=auth_provider)
session = cluster.connect()

In [15]:
# Envoie du dataFrame vers Kafka
serialized_df = data.to_json().encode('utf-8')  # encode the JSON string to bytes
producer.send(topic, value=serialized_df)

# Le dataFrame est ensuite stocké dans la base de données Cassandra
session.execute("CREATE KEYSPACE IF NOT EXISTS your_keyspace WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
session.execute("USE your_keyspace")
session.execute("CREATE TABLE IF NOT EXISTS RSS (title text, pubdate text, link text, description text, PRIMARY KEY(pubDate)) WITH CLUSTERING ORDER BY (pubDate DESC)")

for _, row in data.iterrows():
    session.execute(f"INSERT INTO RSS (title, pubdate, link, description) VALUES ('{row['title']}', '{row['pubDate']}', '{row['link']}', '{row['description']}')")


In [16]:
rows = session.execute("SELECT * FROM RSS")
for row in rows:
    print(row.link)

https://www.actuia.com/?p=42799
https://www.lemonde.fr/international/article/2023/04/06/l-armee-israelienne-bombarde-le-sud-du-liban-apres-des-tirs-de-roquettes_6168538_3210.html
https://www.actuia.com/?p=42895
https://www.actuia.com/?p=42828
https://www.actuia.com/?p=42971
https://www.actuia.com/?p=42850
https://www.actuia.com/?p=42746
https://www.lemonde.fr/planete/live/2023/04/06/contamination-de-l-eau-potable-posez-vos-questions-a-notre-journaliste-stephane-foucart-sur-le-metabolite-du-chlorothalonil_6168544_3244.html
https://www.actuia.com/?p=42948
https://www.actuia.com/?p=42874
https://www.lemonde.fr/societe/article/2023/04/06/la-videosurveillance-intelligente-sera-finalement-experimentee-jusqu-a-six-mois-apres-les-jeux-olympiques_6168540_3224.html
https://www.lemonde.fr/m-styles/article/2023/04/06/jan-jan-van-essche-styliste-globe-trotter_6168460_4497319.html
https://www.actuia.com/?p=43000
https://www.actuia.com/?p=42754
https://www.actuia.com/?p=42992
https://www.actuia.com/?

In [ ]:
#Ferme la connection
def close_connection():
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()

In [8]:
#Suppression des données dans la basse de données
def delete_data():
    session.execute("DESCRIBE KEYSPACES;")
    session.execute("USE your_keyspace;")
    session.execute("DESCRIBE TABLES;")
    session.execute("TRUNCATE RSS")

In [11]:
delete_data()